**Part 1: Web Scraping**

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df = pd.read_html(link)[0]
df.rename(columns = {"Postal code":"PostalCode"}, inplace = True)
df = df[df.Borough != 'Not assigned'] # drop neighborhoods without assigned borough
df.Neighborhood = df.Neighborhood.apply(lambda x: x.replace(" / ", ", ")) # replace default separator

for index, row in df.iterrows():
    if row["Neighborhood"] == "NaN":
        row["Neighborhood"] = row["Borough"]

df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [3]:
# test dataframe against test set
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns = column_names)
test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(df[df["PostalCode"] == postcode], ignore_index = True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


In [4]:
df.shape

(103, 3)

**Part 2: Latitude and Longitude Data**

In [5]:
coordinates = pd.read_csv("http://cocl.us/Geospatial_data", index_col = 0)
neighborhoods = df.merge(coordinates, left_on = "PostalCode", right_on = "Postal Code")
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [6]:
# test data against test data
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df_2 = pd.DataFrame(columns = column_names)
test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df_2 = test_df_2.append(neighborhoods[neighborhoods["PostalCode"] == postcode], ignore_index = True)
    
test_df_2  

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


**Part 3: Clustering**

In [7]:
address = 'Toronto'
geolocator = Nominatim(user_agent = "my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [8]:
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(map_toronto)  
    
map_toronto

In this part, we explore the neighborhoods located specifically in Toronto.

In [9]:
# determine which boroughs have Toronto in their name
borough_names = neighborhoods.Borough.unique()
borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto']

In [10]:
# create a new df with only the above boroughs
toronto = neighborhoods[neighborhoods['Borough'].isin(borough_with_toronto)].reset_index(drop = True)
print(toronto.shape)
toronto.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [11]:
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color ='blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(map_toronto)  
    
map_toronto

In [12]:
CLIENT_ID = 'AREDYM54BSAGERQHZPPGRIF5LZPJTEF3PFLHATZT0JGBZENH' # your Foursquare ID
CLIENT_SECRET = 'SPZANWWMIEFCFOMTERDR2OHFFHK2PEAJVCP02QLMXQ3P50QG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [13]:
radius = 500
LIMIT = 100
venues = []

for lat, long, post, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['PostalCode'], toronto['Borough'], 
                                                  toronto['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id=ZYLC4Q3I000O4R32DVJWJJTOTHCGC4O02TXYEPLDAS211SPQ&client_secret=OPKHF1MTRWKRHVR2DAV0IT1IK2H2XZDXJYTCNHVY5L44T55H&v=20180605 \
     &ll=43.653963,-79.387207&radius=500&limit=100".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [14]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(1716, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Textile Museum of Canada,43.654396,-79.386500,Art Museum
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Cafe Plenty,43.654571,-79.389450,Café
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Sansotei Ramen 三草亭,43.655157,-79.386501,Ramen Restaurant
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Japango,43.655268,-79.385165,Sushi Restaurant


In [15]:
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix = "", prefix_sep = "")
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]
print(toronto_onehot.shape)
toronto_onehot.head()

(1716, 36)


,PostalCode,Borough,Neighborhoods,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop,...,Plaza,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Smoke Shop,Sushi Restaurant,Tapas Restaurant,University,Vegetarian / Vegan Restaurant
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [16]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()
toronto_grouped

,PostalCode,Borough,Neighborhoods,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop,...,Plaza,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Smoke Shop,Sushi Restaurant,Tapas Restaurant,University,Vegetarian / Vegan Restaurant
0,M4E,East Toronto,The Beaches,0.022727,0.068182,0.022727,0.022727,0.022727,0.045455,0.022727,...,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.022727,0.022727
1,M4K,East Toronto,"The Danforth West, Riverdale",0.022727,0.068182,0.022727,0.022727,0.022727,0.045455,0.022727,...,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.022727,0.022727
2,M4L,East Toronto,"India Bazaar, The Beaches West",0.022727,0.068182,0.022727,0.022727,0.022727,0.045455,0.022727,...,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.022727,0.022727
3,M4M,East Toronto,Studio District,0.022727,0.068182,0.022727,0.022727,0.022727,0.045455,0.022727,...,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.022727,0.022727
4,M4N,Central Toronto,Lawrence Park,0.022727,0.068182,0.022727,0.022727,0.022727,0.045455,0.022727,...,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.022727,0.022727
5,M4P,Central Toronto,Davisville North,0.022727,0.068182,0.022727,0.022727,0.022727,0.045455,0.022727,...,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.022727,0.022727
6,M4R,Central Toronto,North Toronto West,0.022727,0.068182,0.022727,0.022727,0.022727,0.045455,0.022727,...,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.022727,0.022727
7,M4S,Central Toronto,Davisville,0.022727,0.068182,0.022727,0.022727,0.022727,0.045455,0.022727,...,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.022727,0.022727
8,M4T,Central Toronto,"Moore Park, Summerhill East",0.022727,0.068182,0.022727,0.022727,0.022727,0.045455,0.022727,...,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.022727,0.022727
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",0.022727,0.068182,0.022727,0.022727,0.022727,0.045455,0.022727,...,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.022727,0.022727


In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
1,M4K,East Toronto,"The Danforth West, Riverdale",Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
2,M4L,East Toronto,"India Bazaar, The Beaches West",Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
4,M4N,Central Toronto,Lawrence Park,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
5,M4P,Central Toronto,Davisville North,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
6,M4R,Central Toronto,North Toronto West,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
7,M4S,Central Toronto,Davisville,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
8,M4T,Central Toronto,"Moore Park, Summerhill East",Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub


Now, we will cluster the neighborhoods by their venues.

In [18]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state = 0).fit(toronto_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

C:\Users\sanja\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [19]:
#create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = toronto.copy()
# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")
print(toronto_merged.shape)
toronto_merged.head() # check the last columns!
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace = True)
toronto_merged

(39, 16)
(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
21,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307,0,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
22,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,0,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
23,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
25,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,0,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
26,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
27,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,0,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
20,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
28,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,0,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub


In [20]:
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster-1],
        fill = True,
        fill_color = rainbow[cluster-1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

In [21]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
21,Central Toronto,0,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
22,West Toronto,0,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
23,Central Toronto,0,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
24,Central Toronto,0,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
25,West Toronto,0,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
26,Central Toronto,0,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
27,Downtown Toronto,0,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
20,Central Toronto,0,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
28,West Toronto,0,Café,Coffee Shop,Art Gallery,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub


In [22]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [24]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
